In [1]:
# $ ipcluster start -n=4
import ipyparallel as ipp

In [2]:
c = ipp.Client()

In [3]:
c.ids

[0, 1, 2, 3]

We, then, make calls happen remotely by turning function calls into calls to apply.  
For instance: we define a lambda function which returns "Hello World !".

In [4]:
c[:].apply_sync(lambda :  "Hello World !")

['Hello World !', 'Hello World !', 'Hello World !', 'Hello World !']

In [5]:
add = lambda x, y: x + y
add(1, 2)

3

In [6]:
def add(x, y):
    return x + y

In [7]:
dview = c[:] 

In [8]:
dview.map_sync(lambda x: x ** 10, range(7))

[0, 1, 1024, 59049, 1048576, 9765625, 60466176]

In [9]:
%%px  
import os
pid = os.getpid()  # every engines will be different
print(pid)

[stdout:0] 19464
[stdout:1] 14952
[stdout:2] 1992
[stdout:3] 12472


In [10]:
dview.map_sync(lambda x: pid, range(7))

[19464, 19464, 14952, 14952, 1992, 1992, 12472]

We see that 7 jobs are distributed in $\{2, 2, 2, 1\}$. You should try different
            numbers of jobs.

In [11]:
dview['a'] = 6
dview['b'] = 7

In [12]:
dview.apply(lambda x: a + b + x, 27)

<AsyncResult: <lambda>>

In [13]:
dview.block = True
dview.apply(lambda x: a + b + x, 27)

[40, 40, 40, 40]

In [14]:
dview.targets

[0, 1, 2, 3]

In [15]:
dview2 = c[1:3]

In [16]:
dview2.targets

[1, 2]

In [17]:
c[::2].execute('c=a+b')

<AsyncResult: execute>

In [18]:
c[1::2].execute('c=a-b')

<AsyncResult: execute>

In [19]:
dview['c'] # shorthand for dview.pull('c', block=True)

[13, -1, 13, -1]

In [20]:
dview.block = False
ar = dview.apply(lambda x: a+b+x, 27)

In [21]:
# Poll to see if the result is ready
ar.ready()

False

In [22]:
# ask for the result
ar.get()

[40, 40, 40, 40]

In [23]:
# define our function
def wait(t):
   import time
   tic = time.time()
   time.sleep(t)
   return time.time() - tic

In [24]:
ar = dview.apply_async(wait, 10)

In [25]:
ar.ready()

False